In [1]:
from examples.client import PG

res = await PG.fetch("SELECT * FROM item")
print(res)

shape: (10, 3)
┌──────────┬───────────┬──────────────┐
│ id       ┆ name      ┆ price        │
│ ---      ┆ ---       ┆ ---          │
│ i64      ┆ str       ┆ decimal[*,2] │
╞══════════╪═══════════╪══════════════╡
│ 1965488  ┆ onion     ┆ 2.99         │
│ 1870644  ┆ bbq       ┆ 6.99         │
│ 69420    ┆ condoms   ┆ 69.42        │
│ 696969   ┆ weed      ┆ 4.20         │
│ 6334     ┆ beans     ┆ 69.99        │
│ 34893    ┆ chalk     ┆ 8.99         │
│ 22334    ┆ poop      ┆ 100.99       │
│ 348374   ┆ yogurt    ┆ 3.99         │
│ 98333829 ┆ chocolate ┆ 4.99         │
│ 83473    ┆ pumpkin   ┆ 12.99        │
└──────────┴───────────┴──────────────┘


In [2]:
import polars as pl

# items = pl.read_csv("../db/csvs/item.csv")
items_to_insert = pl.DataFrame(
    {
        "id": [394789, 3483, 39],
        "name": ["beanie", "foo", "bar"],
        "price": [12.66, 10.0, 20.0],
    }
)

res = await PG.insert(items_to_insert, "item", return_cols={"id"})
print(res)

shape: (3, 1)
┌────────┐
│ id     │
│ ---    │
│ i64    │
╞════════╡
│ 394789 │
│ 3483   │
│ 39     │
└────────┘


In [3]:
res = await PG.insert(items_to_insert, "item", pkey_cols={"id"}, return_cols={"id"})
print(res)

None


In [4]:
res = await PG.fetch(
    "DELETE FROM item WHERE id = ANY(:ids) RETURNING id", ids=[394789, 3483, 39]
)
print(res)

shape: (3, 1)
┌────────┐
│ id     │
│ ---    │
│ i64    │
╞════════╡
│ 394789 │
│ 3483   │
│ 39     │
└────────┘


In [5]:
res = await PG.fetch(
    "SELECT * FROM item WHERE id = ANY(:ids) ORDER BY price LIMIT 10",
    ids=[83473, 348374],
)
print(res)

shape: (2, 3)
┌────────┬─────────┬──────────────┐
│ id     ┆ name    ┆ price        │
│ ---    ┆ ---     ┆ ---          │
│ i64    ┆ str     ┆ decimal[*,2] │
╞════════╪═════════╪══════════════╡
│ 348374 ┆ yogurt  ┆ 3.99         │
│ 83473  ┆ pumpkin ┆ 12.99        │
└────────┴─────────┴──────────────┘


In [6]:
ids_series = pl.Series("id", [83473, 348374])
names_series = pl.Series("name", ["onion", "bbq", "beans"])
attr_dict = {"ids": ids_series, "names": names_series}


query = """--sql
    SELECT * 
    FROM item 
    WHERE 
        id = ANY(:ids) OR
        name = ANY(:names) 
    ORDER BY price 
    LIMIT 10
    """

await PG.fetch(
    query,
    **attr_dict,
)

id,name,price
i64,str,"decimal[*,2]"
1965488,"""onion""",2.99
348374,"""yogurt""",3.99
1870644,"""bbq""",6.99
83473,"""pumpkin""",12.99
6334,"""beans""",69.99
